In [1]:
import pandas as pd

# Abrimos el documento que hemos generado al separar por coding y non coding
with open("hnoncoding.txt", 'r') as f:
    
    #Creamos el dataframe necesario
    columns=["ID"]
    tableid2=pd.DataFrame(columns=columns)
    
    #Creamos una lista en la que guardamos cada linea del documento
    file=f.readlines()
    
    #lista vacia para ir guardando los identificadores de cada transcrito
    ID=[]
    
    #Iteramos sobre la lista, el identificador esta separado del resto de información por tabuladores, de esta forma podemos guardar
    #quedarnos solo con el identificador
    for i in file:
        fin=i.find("\t")
        ID.append(i[0:fin])

In [2]:
# Guardamos en el dataframe los ID
tableid2["ID"]=ID

In [3]:
#Abrimos el fasta con todos los transcritos generados en el assembly, el objetivo es guardar en cada entrada del diccionario
# el identificador y la secuencia correspondiente.
with open("hard_filtered_transcripts.fasta", 'r') as f:
    dic={}
    file=f.readlines()
    count=0
    
    #al ser un fasta el identificador viene siempre precedido por ">"
    for i in file:
        if i[0]==">":
            fin=i.find("\n")
            name=i[0:fin]
            dic[name]=[]
        else:
            dic[name].append(i.replace('\n',''))

In [4]:
#Hemos guardado en el diccionario las secuencias en forma de lista, de esta forma las juntamos todas en una unica string
for i in dic:
    dic[i]="".join(dic[i])

In [5]:
#Ahora tenemos que guardar en una lista los transcritos que sean noncoding, al iterar sobre los identificadores del dataframe
#que pertenecen a secuencias noncoding, podemos buscar dicho identificador en el diccionario y guardar esa secuencai en una lista
#que posteriormente guardaremos en el dataframe
noncoding=[]
for i in tableid2["ID"]:
    name=">"+i
    noncoding.append(dic[name])

In [6]:
tableid2["Sequence"]=noncoding

In [7]:
tableid2.head()

,ID,Sequence
0,NODE_56_length_16476_cov_24.731756_g17_i1,AGATCTGGGAAGCGGCAGCCACATCGGGGACTGGTGGTCCGCTCCC...
1,NODE_60_length_16326_cov_476.574971_g20_i0,AGCAGAGAATTATATGTGCTTTTGTAAATAAATTAAAAGATCTTGT...
2,NODE_66_length_16238_cov_471.756573_g20_i4,AGCAGAGAATTATATGTGCTTTTGTAAATAAATTAAAAGATCTTGT...
3,NODE_82_length_14978_cov_171.135793_g32_i0,CGACCACCGAAAAGAATGTTAGTGTAGAAGCAAAGATAGTAGACAG...
4,NODE_87_length_14644_cov_249.505044_g35_i0,GCCAGTTCTGTAAGGTGACTGTACTACCATATGGTCAAATGATTAA...


In [8]:
#De esta forma guardamos en el formato fasta nuestras secuencias noncoding.
with open("HNonCodingGenes.fasta", 'w') as f:
    for i,j in zip(tableid2["Sequence"],tableid2["ID"]):
        name=">"+j
        f.write(name)
        f.write("\n")
        f.write(i)
        f.write("\n")

In [9]:
#Comprobar que el número de transcritos sea igual que en el paso de CPC2
tableid2.count(axis=1)

0         2
1         2
2         2
3         2
4         2
         ..
616147    2
616148    2
616149    2
616150    2
616151    2
Length: 616152, dtype: int64